In [ ]:
reload_lamb()

# Ad-hoc polymorphism via disjunctive types

The lambda notebook supports ad-hoc polymorphism via what are called disjunctive types.  A disjunctive type is effectively a set of types that unifies with another type if either (i) that type is disjunctive and there is overlap, or (ii) that type is a member of the disjunctive type.  If the result of unification would be a singleton, the resulting type is the unique member of that singleton.  Disjunctive types are written with square brackets and `|` as the separator.  They can be written recursively, but will be interpreted with that recursion flattened out.

**examples**
1. `[e|t]` and `[e|t]` unify to `[e|t]` (reflexivity)
2. Disjunctive types are normalized alphabetically, so for example `[t|e]` is normalized as `[e|t]`.  This ensures symmetry.
3. `[e|t|<e,t>]` and `[e|t|n]` unify to `[e|t]`
4. `[e|t]` and `e` unify to `e`.
5. `[e|t]` and `[t|n]` unify to `t`.
6. `[e]` is invalid.  So is `[e|e]`, etc.
7. `[e|e|t]` is interpreted as just `[e|t]`.
8. `[e|[n|t]]` is interpreted as just `[e|n|t]`

One further constraint is that type variables (or types using them) cannot be disjoined.

In [ ]:
unify(tp("[e|t]"), tp("[e|t]"))

In [ ]:
tp("[t|e]")

In [ ]:
unify(tp("[e|t|<e,t>]"), tp("[e|t|n]"))

In [ ]:
unify(tp("[e|t]"), tp("e"))

In [ ]:
unify(tp("[e|t]"), tp("[t|n]"))

In [ ]:
try:
    tp("[e]")
except types.TypeParseError as e:
    print(e)

In [ ]:
try:
    tp("[e|e]")
except types.TypeParseError as e:
    print(e)

In [ ]:
tp("[e|e|t]")

In [ ]:
tp("[e|[n|t]]")

In [ ]:
unify(tp("[e|t]"), tp("n"))

In [ ]:
unify(tp("[<e,t>|<e,n>]"), tp("X"))

In [ ]:
unify(tp("[<e,t>|<e,n>|<n,t>]"), tp("<X,t>"))

In [ ]:
unify(tp("[<e,t>|<e,n>|<n,<e,t>>]"), tp("<X,<Y,t>>"))

You can think of disjunctive types as tools for characterizing finite sets of non-polymorphic types.  In some cases there may be multiple ways of describing the same set, and unification will produce a coherent result:

In [ ]:
unify(tp("<e,[t|n]>"), tp("[<e,t>|<e,n>]"))

In [ ]:
tp("[<e,t>|<e,n>]").factor_functional_types() # convert to a functional type.  Will return None if no such conversion is possible.

In [ ]:
tp("[<e,t>|<n,t>]").factor_functional_types()

(It isn't guaranteed that there is a canonical principal type for these cases, though there is for simple cases -- but it is guaranteed that unification will not return `None`.)

## Using disjunctive types in expressions

Disjunctive types can be straightforwardly used as term types, and all appropriate type inference should happen as expected.  This is the most typical use in linguistics, where someone might want to express that a (not-further-defined) constant is polymorphic in some way.

In [ ]:
%%lamb 
||equals|| = L x_[e|n] : Equivalent(x)
x = x_e

In [ ]:
equals.content(x) # forces narrowing of the argument type

In [ ]:
%te P_<[e|n],t>(x_[n|t]) # forces narrowing of both types

In [ ]:
%te (L x_[e|n] : P_<[e|n],t>(x))(C_n) # forces narrowing including of both x and, indirectly, the predicate's type

While you can't use variable types in disjunctive types, they will unify with variable types correctly:

In [ ]:
te("(L x_[e|n] : P_<[e|n|t],t>(x))(C_X)")

The other way to use disjunctive types is via a `Disjunctive` expression.  This gives, basically, a disjunctive formula, where type inference will choose between expressions depending on type.  You can think of this (in a roundabout way) as something like a `dict`/hashtable mapping types to expressions.  This may help understand some odd corner cases -- `Disjunctive` expressions have to ensure that every non-disjunctive type is mapped to a unique expression.  A disjunctive expression displays as the function Disjunctive, with the type annotated as a superscript.

In [ ]:
x = %te Disjunctive(A_e, B_n, C_t)
x

Adjusting the type, either explicitly or implicitly, can narrow down the formula, resulting in either another Disjunctive type, or just some TypedExpr.  (Or `None`, if the adjustment fails.)

In [ ]:
x.try_adjust_type(tp("e"))

In [ ]:
x.try_adjust_type(tp("[e|t]"))

In [ ]:
x.try_adjust_type(tp("[e|t]")).try_adjust_type(tp("n")) # leads to None

In [ ]:
te("Disjunctive(x_e, y_[<e,t>|n], z_t)").try_adjust_type(tp("[e|t]"))

Disjuncts most have unique types, or more generally, must lead deterministically to unique formulas for every type refinement.  So the following cases are out.  The second, more complicated case, would lack a unique refinement at type `e`.

In [ ]:
%te Disjunctive(x_e, y_e)

In [ ]:
%te Disjunctive(x_e, y_e, z_[e|t])

While disjunctive types cannot recurse, Disjunction expressions can recurse or involve multiple disjunctive types, as long as they obey the other type constraints.  (There still must be a unique refinement.)  Type adjustment will result in flattening of Disjunctions.

In [ ]:
r = %te Disjunctive(x_e, Disjunctive(y_n, z_t))
r

In [ ]:
r.try_adjust_type(tp("n"))

In [ ]:
r.try_adjust_type(tp("[e|n]"))

In [ ]:
r.try_adjust_type(tp("[n|t]"))

In [ ]:
%te Disjunctive(x_e, y_[<e,t>|n])

Quite complex things are possible with disjunctive types, once you combine them with other tools.  Keep in mind, though, that type refinements on variables must be stable across all instances of that variable's use in the same scope.

In [ ]:
f = %te L x_e : Disjunctive(x_[e|n], False_t)
f

In [ ]:
f.type

In [ ]:
%te reduce (L x_e : Disjunctive(x_[e|n], y_t))(A_e)

In [ ]:
%te reduce (L x_[e|t] : P(x) & x)((L x_e : Disjunctive(x_[e|n], y_t))(A_e))

In [ ]:
%te reduce (L x_e : Disjunctive(x_[e|n], False_t))(A_e) & P_<e,t>((L x_e : Disjunctive(x_[e|n], False_t))(A_e))

Reduction of disjunctive functional types works.

In [ ]:
f = %te Disjunctive((L f_<e,t> : f), (L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)))
f

In [ ]:
f(te("P_<e,t>")).reduce_all()

In [ ]:
%te reduce (Disjunctive((L x_e : x), P_<e,t>))(A_e)

If the reduction for some disjunct wouldn't work (either the types don't work or the disjunct isn't a function) the reduction will simply drop that result.  If there are no disjuncts where the combination works, you get a type mismatch.

In [ ]:
%te reduce (Disjunctive((L x_e : x), P_<n,t>))(A_e)

In [ ]:
%te reduce (Disjunctive((L x_e : x), P_n))(A_e)

In [ ]:
%te reduce (Disjunctive((L x_n : x), P_n))(A_e)

To expand on the pitfall mentioned above, polymorphic types can only be refined to more specific principle types, and for bound variables, these refinements must be consistent across all instances.  So for example, the following will generate a type error, because it proposes two inconsistent refinements for `x`.

In [ ]:
%te L x_[e|n] : Disjunctive(x_e, x_n)

This also goes for cases where the refinements are triggered indirectly via type inference, such as the following.  The error messages for these cases may be somewhat oblique, depending on circumstances (though this one is clear).

In [ ]:
%te L x_[e|n] : Disjunctive(P_<e,e>(x), Q_<n,t>(x))